# PJN L2

In [125]:
import numpy as np
import random
import itertools
from collections import defaultdict
import math
import os
from itertools import permutations
import time
from scipy.stats import mode
import re
import string

## Zadanie 1. (3+1+Xp) 
Napisz program, który ustala autora zdania. Autorów jest trójka: Prus, Orzeszkowa, Sienkiewicz (POS), na SKOSie znajdziesz odpowiednie dane uczące. Powinieneś je podzielić na część uczącą i walidacyjną. Część X punktacji zależeć będzie od tego, jak Twój program wypadnie na tle innych programów dla danych testowych (które pojawią się później na SKOSie). 

Dwie podstawowe metody są następujące:

a) Naive Bayes (będzie na wykładzie 4, powinieneś użyć przynajmniej jednej cechy, która nie jest słowem)

b) Utworzenie trzech modeli językowych i wybór tego, który daje najepszy wynik na klasyfikowanym zdaniu.

(Za sprawdzenie dwóch podejść jest punkt premiowy)

In [2]:
P_words = defaultdict(lambda:1)
O_words = defaultdict(lambda:1)
S_words = defaultdict(lambda:1)
P_lens = defaultdict(lambda:1)
O_lens = defaultdict(lambda:1)
S_lens = defaultdict(lambda:1)

P_valid_sentences = []
O_valid_sentences = []
S_valid_sentences = []

In [3]:
names = ['prusa','orzeszkowej','sienkiewicza']
words_dicts = [P_words,O_words,S_words]
lens_dicts = [P_lens,O_lens,S_lens]
valid_lists = [P_valid_sentences,O_valid_sentences,S_valid_sentences]

random.seed(42)
VALID_PROB = 0.15

for i,name in enumerate(names):
    words_dict = words_dicts[i]
    lens_dict = lens_dicts[i]
    valid_list = valid_lists[i]
    with open(f'./dane_pozytywistyczne/korpus_{name}.txt', 'r', encoding='utf-8') as f:
        for l in f:
            if len(l[:-1]):
                for sentence in l[:-1].split('. '):
    
                    if random.random() < VALID_PROB:
                        valid_list.append(sentence)
                        continue
                    for w in sentence.split(' '):
                        words_dict[w.lower()]+=1
                    lens_dict[len(sentence.split(' '))]+=1

In [4]:
P_words_c, O_words_c, S_words_c = map(lambda e: sum(list(e.values())), words_dicts)
P_lens_c, O_lens_c, S_lens_c = map(lambda e: sum(list(e.values())), lens_dicts)

In [5]:
N_sents_P, N_sents_O, N_sents_S = map(lambda e: sum(list(e.values()))-len(list(e.values())), lens_dicts)
total_sents_sum = N_sents_P + N_sents_O + N_sents_S
totalPP,totalPO,totalPS = math.log(N_sents_P/total_sents_sum), math.log(N_sents_O/total_sents_sum), math.log(N_sents_S/total_sents_sum)

In [6]:
def predict(s):
    PP,PO,PS = .0,.0,.0
    for w in s:
        w = w.lower()
        PP+=math.log(P_words[w]/P_words_c)
        PS+=math.log(S_words[w]/S_words_c)
        PO+=math.log(O_words[w]/O_words_c)
        
    s_len = len(s)
    PP+=math.log(P_lens[s_len]/P_lens_c)
    PS+=math.log(S_lens[s_len]/S_lens_c)
    PO+=math.log(O_lens[s_len]/O_lens_c)
    
    PP,PS,PO = PP+totalPP,PS+totalPS,PO+totalPO
    
    return np.argmax(np.array([PP,PO,PS]))

In [7]:
confustion_matrix = np.zeros((3,3))
for i, valid_set in enumerate(valid_lists):
    for sent in valid_set:
        s = sent.split(' ')
        c = predict(s)
        confustion_matrix[c,i] += 1

print(confustion_matrix)
np.sum(np.diag(confustion_matrix))/np.sum(confustion_matrix,axis=None)

[[1513.  235.  160.]
 [  82. 1288.   44.]
 [ 474.  413.  659.]]


0.7107641741988496

### Tests

In [8]:
test_files = os.listdir('./dane_pozytywistyczne/testy1/')
test_sentences = []
test_labels = []
for test_file in test_files:
    if 'prusa' in test_file:
        label = 0
    elif 'orzeszkowej' in test_file:
        label = 1
    else:
        label = 2
        
    with open('./dane_pozytywistyczne/testy1/'+test_file, 'r', encoding='utf-8') as f:
        for l in f:
            if len(l[:-1]):
                for sentence in l[:-1].split('. '):
                    test_sentences.append(sentence)
                    test_labels.append(label)

In [9]:
confustion_matrix = np.zeros((3,3))
for i,sentence in enumerate(test_sentences):
    label = test_labels[i]
    pred = predict(sentence.split(' '))
    confustion_matrix[pred,label] += 1
    
print(confustion_matrix)
np.sum(np.diag(confustion_matrix))/np.sum(confustion_matrix,axis=None)

[[1099.   83.  705.]
 [  65.  200.  105.]
 [ 453.  398. 1108.]]


0.5709203036053131

## Zadanie 2. (3+Xp) 
W zadaniu tym powinieneś uporządkować (spermutować) ciąg słów, żeby utworzył zdanie. W stosunku do poprzedniej listy mamy następujące różnice:

a) powinieneś wykorzystać tagi słów (na przykład z pliku supertags.txt, link na SKOSie)

b) powinieneś je połączyć ze zwykłymi statystykami bigramowymi (lub, opcjonalnie, z sufiksami)

c) powinieneś wyodrębnić z danych uczących część walidacyjną i dobrać parametry łączenia modeli
bazujących na słowach i bazujących na tagach.

Sposób oceny będzie taki sam, jak w przypadku zadania z P1. Wybór zdań do oceny pojawi się
przed zajęciami (będą to zdania zawierające od 4 do 8 tokenów, wybrane losowo z części testowej
korpusu PolEval)

In [191]:
def add_bigram(s1,s2,n=1):
    if s1 not in BIGRAMS_DICT:
        BIGRAMS_DICT[s1] = {s2:n}
    else:
        if s2 in BIGRAMS_DICT[s1]:
            BIGRAMS_DICT[s1][s2] += n
        else:
            BIGRAMS_DICT[s1][s2] = n

In [10]:
BIGRAMS_DICT = {}
with open('poleval_2grams.txt','r',encoding='UTF-8') as f:
    for line in f:
        n,s1,s2 = line.lower()[:-1].split(' ')
        n = int(n)
        if n >= 10:
            add_bigram(s1,s2)

In [11]:
TRIGRAMS_DICT = {}
with open('poleval_3grams.txt','r',encoding='UTF-8') as f:
    for line in f:
        line = line.lower()[:-1].split(' ')
        if len(line) != 4:
            continue
        n,s1,s2,s3 = line
        n = int(n)
        if n >= 10:
            if (s1,s2) not in TRIGRAMS_DICT:
                TRIGRAMS_DICT[(s1,s2)] = {s3:n}
            else:
                TRIGRAMS_DICT[(s1,s2)][s3] = n

In [12]:
def P(w,w1=None,w2=None,l1=0.05,l2=0.11,l3=0.84):
    N = 4000000
    pw,pww,pwww = 1/N,0,0
    
    if w in BIGRAMS_DICT:
        s = sum(BIGRAMS_DICT[w].values())
        pw = s/N
        
    if w1 is not None and w1 in BIGRAMS_DICT:
        D = BIGRAMS_DICT[w1]
        if w in D:
            s = sum(D.values())
            pww = D[w]/s
                
    if w2 is not None and w1 is not None and (w2,w1) in TRIGRAMS_DICT:
        D = TRIGRAMS_DICT[(w2,w1)]
        if w in D:
            s = sum(D.values())
            pwww = D[w]/s
    
    return math.log(l1*pw + l2*pww + l3*pwww)

In [13]:
def compare_perms(p1,p2):
    for i,w in enumerate(p1):
        if w != p2[i]:
            return False
    return True        

In [14]:
def order_words(tokens,f=P,l1=0.05,l2=0.11,l3=0.84):
    results = []
    tokens = [t.lower() for t in tokens]
    for perm in permutations(tokens):
        p = 0.0
        p += f(perm[0])
        p += f(perm[1], perm[0])
        for i in range(2,len(tokens)):
            p += f(perm[i], perm[i-1], perm[i-2], l1, l2, l3)
        results.append((p,perm))
    sorted_result = sorted(results,key=lambda e:e[0],reverse=True)
    for i,(s,perm) in enumerate(sorted_result):
        if compare_perms(tokens,perm):
            return 1/(i+1)
    return 0

In [287]:
def test(f=P,l1=0.05,l2=0.11,l3=0.84):
    sum_ = 0.0
    N = len(ZDANIA)
    ones = 0
    for i,s in enumerate(ZDANIA):
#         if len(s) >= 8:
#             print(f'Zdanie {i+1}/{N}, wynik: skipped')
#             continue
        score = order_words(s,f,l1,l2,l3)
        sum_ += score
        if int(score) == 1:
            ones+=1
        print(f'Zdanie {i+1}/{N}, wynik: {score}')
    print()
    print(f'Wynik całkowity: {sum_/N}, ilość poprawnych wyników: {ones}/{N}')

In [47]:
test()

Zdanie 1/65, wynik: 0.01818181818181818
Zdanie 2/65, wynik: 0.14285714285714285
Zdanie 3/65, wynik: 0.14285714285714285
Zdanie 4/65, wynik: 0.006802721088435374
Zdanie 5/65, wynik: skipped
Zdanie 6/65, wynik: 0.004878048780487805
Zdanie 7/65, wynik: 0.004784688995215311
Zdanie 8/65, wynik: 0.3333333333333333
Zdanie 9/65, wynik: 0.024390243902439025
Zdanie 10/65, wynik: 0.043478260869565216
Zdanie 11/65, wynik: 0.010416666666666666
Zdanie 12/65, wynik: 0.029411764705882353
Zdanie 13/65, wynik: 0.011627906976744186
Zdanie 14/65, wynik: 0.3333333333333333
Zdanie 15/65, wynik: 1.0
Zdanie 16/65, wynik: 0.008264462809917356
Zdanie 17/65, wynik: 1.0
Zdanie 18/65, wynik: 0.04
Zdanie 19/65, wynik: 0.004878048780487805
Zdanie 20/65, wynik: 1.0
Zdanie 21/65, wynik: 0.5
Zdanie 22/65, wynik: 1.0
Zdanie 23/65, wynik: 0.06666666666666667
Zdanie 24/65, wynik: 0.16666666666666666
Zdanie 25/65, wynik: 0.047619047619047616
Zdanie 26/65, wynik: 0.25
Zdanie 27/65, wynik: 1.0
Zdanie 28/65, wynik: 0.05263157

In [288]:
test()

Zdanie 1/65, wynik: 0.017857142857142856
Zdanie 2/65, wynik: 0.07692307692307693
Zdanie 3/65, wynik: 0.1111111111111111
Zdanie 4/65, wynik: 0.006802721088435374
Zdanie 5/65, wynik: 0.0009784735812133072
Zdanie 6/65, wynik: 0.004878048780487805
Zdanie 7/65, wynik: 0.004784688995215311
Zdanie 8/65, wynik: 1.0
Zdanie 9/65, wynik: 0.02564102564102564
Zdanie 10/65, wynik: 0.07692307692307693
Zdanie 11/65, wynik: 0.008849557522123894
Zdanie 12/65, wynik: 0.030303030303030304
Zdanie 13/65, wynik: 0.011627906976744186
Zdanie 14/65, wynik: 1.0
Zdanie 15/65, wynik: 1.0
Zdanie 16/65, wynik: 0.008130081300813009
Zdanie 17/65, wynik: 0.2
Zdanie 18/65, wynik: 0.04
Zdanie 19/65, wynik: 0.005025125628140704
Zdanie 20/65, wynik: 0.5
Zdanie 21/65, wynik: 1.0
Zdanie 22/65, wynik: 1.0
Zdanie 23/65, wynik: 0.06666666666666667
Zdanie 24/65, wynik: 0.16666666666666666
Zdanie 25/65, wynik: 0.058823529411764705
Zdanie 26/65, wynik: 1.0
Zdanie 27/65, wynik: 0.25
Zdanie 28/65, wynik: 0.05263157894736842
Zdanie 2

### Tags

In [16]:
TAGS = {}
with open('./supertags.txt','r',encoding='utf-8') as f:
    for line in f:
        w,t = line[:-1].split(' ')
        TAGS[w] = t

In [189]:
def add_bigram_tag(t1, t2, n=1):
    if t1 not in BIGRAMS_TAGS:
        BIGRAMS_TAGS[t1] = {t2:n}
    else:
        if t2 in BIGRAMS_TAGS[t1]:
            BIGRAMS_TAGS[t1][t2] += n
        else:
            BIGRAMS_TAGS[t1][t2] = n

In [17]:
BIGRAMS_TAGS = {}

with open('poleval_2grams.txt','r',encoding='UTF-8') as f:
    for line in f:
        n,s1,s2 = line.lower()[:-1].split(' ')
        if s1 not in TAGS or s2 not in TAGS:
            continue
        n,t1,t2 = int(n), TAGS[s1], TAGS[s2]
        add_bigram_tag(t1,t2,n)

In [18]:
def P2(w,w1=None,w2=None,l1=0.2,l2=0.6,l3=0.2):
    N = 40000000
    pw,pww,pt = 1/N,0,0
    
    if w in BIGRAMS_DICT:
        s = sum(BIGRAMS_DICT[w].values())
        pw = s/N
        
    if w1 is not None and w1 in BIGRAMS_DICT:
        D = BIGRAMS_DICT[w1]
        if w in D:
            s = sum(D.values())
            pww = D[w]/s
            
    if w1 is not None and w in TAGS and w1 in TAGS:
        t,t1 = TAGS[w], TAGS[w1]
        if t1 in BIGRAMS_TAGS:
            D = BIGRAMS_TAGS[t1]
            if t in D:
                s = sum(D.values())
                pt = D[t]/s
                
    return math.log(l1*pw + l2*pww + l3*pt)

In [93]:
tests = []
i = 0
with open('polish_corpora.txt', 'r', encoding='utf-8') as f:
    for line in f:
        if random.random() < 0.1:
            s = line[:-1].split(' ')
            if len(s) > 2 and len(s) < 7:
                tests.append(s)
                i += 1
        if i>20:
            break

In [78]:
tests

[['Odkryta', '10', 'marca', '2002', 'roku', '.'],
 ['Miejscowość', 'początkowo', 'nosiła', 'nazwę', 'Astapowo', '.'],
 ['Kto', 'jest', 'za', 'jej', 'odrzuceniem', '?'],
 ['Jaką', 'sztuką', 'walki', 'jest', 'kick-boxing', '?'],
 ['Klub', 'Platformy', 'Obywatelskiej', 'poprze', 'poprawkę', '2.'],
 ['Oczekiwałbym', 'nowelizacji', 'w', 'tym', 'zakresie', '.'],
 ['Dziękuję', 'bardzo', 'za', 'to', 'wyjaśnienie', '.'],
 ['Zmarł', '14', 'kwietnia', '1917', 'r.'],
 ['Próba', 'zabójstwa', 'została', 'jednak', 'udaremniona', '.'],
 ['Wiersze', 'z', 'l.', '1832-1835', '4.'],
 ['Czy', 'w', 'świetle', 'nowych', 'kodyfikacji', '...'],
 ['W', 'dniu', '3', 'grudnia', '2004', 'r.'],
 ['Następna', 'sprawa', '-', 'gospodarstwo', 'rodzinne', '.'],
 ['Samorząd', 'terytorialny', 'musi', 'pozostać', 'samorządem', '.'],
 ['Biskupstwo', 'jest', 'sufraganią', 'archidiecezji', 'Hankou', '.'],
 ['Odkryta', '7', 'maja', '2002', 'roku', '.'],
 ['Rzeźba', 'nie', 'doczekała', 'się', 'kopii', '.'],
 ['Raport', 'Marland

In [94]:
res = []
for i in np.arange(0.05,1,0.05):
    for j in np.arange(i+0.05,1,0.05):
        l1,l2,l3 = i,j-i,1-j
        sum_ = 0.0
        N = len(tests)
        for s in tests:
            sum_ += order_words(s,P2,l1,l2,l3)
        res.append(((sum_/N),(l1,l2,l3)))

In [113]:
sorted(res, key=lambda e:e[0], reverse=True)

[(0.5623431877111825, (0.05, 0.9000000000000002, 0.04999999999999971)),
 (0.5419370658594178, (0.1, 0.8499999999999999, 0.050000000000000155)),
 (0.5396012986943438, (0.05, 0.8500000000000002, 0.09999999999999976)),
 (0.5345999548786162,
  (0.15000000000000002, 0.7499999999999999, 0.10000000000000009)),
 (0.5341227260693794,
  (0.15000000000000002, 0.7999999999999997, 0.050000000000000266)),
 (0.5327469624883419, (0.1, 0.7999999999999998, 0.1000000000000002)),
 (0.5315463106361626, (0.2, 0.7, 0.10000000000000009)),
 (0.5296609424682641,
  (0.15000000000000002, 0.6999999999999998, 0.15000000000000013)),
 (0.5296463914404114, (0.3, 0.5999999999999999, 0.1000000000000002)),
 (0.5287552592195653, (0.2, 0.6499999999999999, 0.15000000000000013)),
 (0.5281715261255678, (0.1, 0.7499999999999999, 0.15000000000000013)),
 (0.5273882304743166, (0.3, 0.5499999999999998, 0.15000000000000013)),
 (0.5270272540804748,
  (0.35000000000000003, 0.5499999999999998, 0.10000000000000009)),
 (0.52682423180199

In [115]:
test(f=P2,l1=0.05,l2=0.9,l3=0.05)

Zdanie 1/65, wynik: 1.0
Zdanie 2/65, wynik: 0.5
Zdanie 3/65, wynik: 0.1111111111111111
Zdanie 4/65, wynik: 0.5
Zdanie 5/65, wynik: skipped
Zdanie 6/65, wynik: 0.0053475935828877
Zdanie 7/65, wynik: 0.00267379679144385
Zdanie 8/65, wynik: 0.008547008547008548
Zdanie 9/65, wynik: 0.1111111111111111
Zdanie 10/65, wynik: 0.014084507042253521
Zdanie 11/65, wynik: 0.007518796992481203
Zdanie 12/65, wynik: 0.006493506493506494
Zdanie 13/65, wynik: 0.015151515151515152
Zdanie 14/65, wynik: 0.1
Zdanie 15/65, wynik: 0.06666666666666667
Zdanie 16/65, wynik: 0.2
Zdanie 17/65, wynik: 0.07142857142857142
Zdanie 18/65, wynik: 0.058823529411764705
Zdanie 19/65, wynik: 0.5
Zdanie 20/65, wynik: 1.0
Zdanie 21/65, wynik: 0.03225806451612903
Zdanie 22/65, wynik: 0.058823529411764705
Zdanie 23/65, wynik: 0.004032258064516129
Zdanie 24/65, wynik: 0.05555555555555555
Zdanie 25/65, wynik: 0.06666666666666667
Zdanie 26/65, wynik: 0.3333333333333333
Zdanie 27/65, wynik: 0.14285714285714285
Zdanie 28/65, wynik: 1

In [289]:
test(f=P2,l1=0.05,l2=0.9,l3=0.05)

Zdanie 1/65, wynik: 1.0
Zdanie 2/65, wynik: 0.5
Zdanie 3/65, wynik: 0.1111111111111111
Zdanie 4/65, wynik: 0.5
Zdanie 5/65, wynik: 0.023255813953488372
Zdanie 6/65, wynik: 0.0053475935828877
Zdanie 7/65, wynik: 0.00267379679144385
Zdanie 8/65, wynik: 0.008547008547008548
Zdanie 9/65, wynik: 0.1111111111111111
Zdanie 10/65, wynik: 0.014084507042253521
Zdanie 11/65, wynik: 0.007518796992481203
Zdanie 12/65, wynik: 0.006493506493506494
Zdanie 13/65, wynik: 0.015151515151515152
Zdanie 14/65, wynik: 0.1
Zdanie 15/65, wynik: 0.06666666666666667
Zdanie 16/65, wynik: 0.2
Zdanie 17/65, wynik: 0.07692307692307693
Zdanie 18/65, wynik: 0.058823529411764705
Zdanie 19/65, wynik: 0.5
Zdanie 20/65, wynik: 1.0
Zdanie 21/65, wynik: 0.03225806451612903
Zdanie 22/65, wynik: 0.058823529411764705
Zdanie 23/65, wynik: 0.004032258064516129
Zdanie 24/65, wynik: 0.05555555555555555
Zdanie 25/65, wynik: 0.06666666666666667
Zdanie 26/65, wynik: 0.3333333333333333
Zdanie 27/65, wynik: 0.14285714285714285
Zdanie 28

## Zadanie 3. (4p) 
W zadaniu tym powinieneś losować zdania o słowach z identyczną charakterystyką gramatyczną jak zdanie wejściowe). Przykładowo dla zdania: 

Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę.

wynikiem mogłoby być

Gruby Stefan przeczytał we wczorajszej gazecie starannie przygotowaną analizę.

Zgodność gramatyczną sprawdzamy za pomocą tagów z pliku supertags. Przyjmijmy, że słowo s niewystępujące w tym pliku ma opis gramatyczny (’^’ + s)[-3:]. Powinieneś korzystać ze
statystyk unigramowych.

In [19]:
SUFIX_TAGS = {}
with open('./supertags.txt','r',encoding='utf-8') as f:
    for line in f:
        w,t = line[:-1].split(' ')
        sufix = w[-3:]
        if sufix not in SUFIX_TAGS:
            SUFIX_TAGS[sufix] = [t]
        else:
            SUFIX_TAGS[sufix].append(t)

In [20]:
for sufix in SUFIX_TAGS.keys():
    SUFIX_TAGS[sufix] = mode(SUFIX_TAGS[sufix])[0][0]

In [21]:
TAGS_TO_WORDS = {}
with open('./supertags.txt','r',encoding='utf-8') as f:
    for line in f:
        w,t = line[:-1].split(' ')
        if t in TAGS_TO_WORDS:
            TAGS_TO_WORDS[t].append(w)
        else:
            TAGS_TO_WORDS[t] = [w]

In [22]:
def get_tag(w):
    if w in TAGS:
        return TAGS[w]
    elif w[-3:] in SUFIX_TAGS:
         return SUFIX_TAGS[w[-3:]]
    return random.choice(list(TAGS_TO_WORDS.keys()))

In [23]:
def generate_similar(s):
    res = []
    for w in s.split(' '):
        w = w.lower()
        t = get_tag(w)
        res.append(random.choice(TAGS_TO_WORDS[t]))
        
    return ' '.join(res)

In [24]:
generate_similar('Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę')

'naporowy manfred przeczyścił pomiędzy niedelikatnej spółce eliptycznie udomowioną pagodę'

## Zadanie 4. (3p) 
Dodaj statystyki bigramowe do powyższego zadania. Postaraj się, by jak najrzadziej zdażały się sytuacje, w których musisz losować posługując się unigramami. Zaznaczaj znakiem "|" każdą taką nieciągłość.

In [25]:
def generate_similar2(s):
    res = []
    words = s.split(' ')
    lw = ''
    
    w = words[0]
    t = get_tag(w)
    options = TAGS_TO_WORDS[t]
    filtered_options = [w for w in options if w in BIGRAMS_DICT]
    if len(filtered_options):
        lw = random.choice(filtered_options)
    else:
        lw = random.choice(options)
    res.append(lw)

    for w in words[1:]:
        w = w.lower()
        t = get_tag(w)
        if lw in BIGRAMS_DICT:
            D = BIGRAMS_DICT[lw]
            options = np.array([[k,v] for k,v in D.items() if get_tag(k)==t])

            if len(options):
                words,weights = options[:,0],list(map(int,options[:,1]))
                new_w = random.choices(words,weights=weights,k=1)[0]
                res.append(new_w)
                continue
            
        res.append('|')
        res.append(random.choice(TAGS_TO_WORDS[t]))
            
    return ' '.join(res)

In [26]:
generate_similar2('Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę')

'zawiązywały | konrad | pokiereszował | na | nieprzekonywającej | bełzie | wzajem | obelkowaną | taterniczkę'

In [87]:
def generate_similar3(s, K=1000):
    res = ''
    words = s.split(' ')
    tags = list(map(get_tag, words))
    options = list(map(lambda t: TAGS_TO_WORDS[t], tags))

    current_sentences = [[w] for w in options[0]]
    next_sentences = []

    i = 1
    while i < len(words):
        if len(current_sentences) > K:
            random.shuffle(current_sentences)
            current_sentences = current_sentences[:K]
            
        searched = {}
        for sent in current_sentences:

            if sent[-1] not in BIGRAMS_DICT:
                continue
                
            if sent[-1] in searched:
                for w in searched[sent[-1]]:
                    new_sent = sent[:]
                    new_sent.append(w)
                    next_sentences.append(new_sent)
                continue

            D = BIGRAMS_DICT[sent[-1]]
            sw = []
            for pnw in options[i]:
                if pnw in D:
                    new_sent = sent[:]
                    new_sent.append(pnw)
                    sw.append(pnw)
                    next_sentences.append(new_sent)
            if len(sw)>0:
                searched[sent[-1]] = sw

        if len(next_sentences) == 0:
            choosen_sent = random.choice(current_sentences)
            res += ' ' + ' '.join(choosen_sent) + ' | '
            current_sentences = [[w] for w in options[i]]
        else:
            current_sentences = next_sentences
            next_sentences = []
        i += 1
    choosen_sent = random.choice(current_sentences)
    res += ' ' + ' '.join(choosen_sent)
    
    return res[1:]

In [88]:
generate_similar3('Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę')

'wyziębiały |  ustawodawca wyznaczył w niewielkiej mierze bardzo udaną współpracę'

In [76]:
res

' rozwydrzały |  aktor otrzymał w niepodległej polsce niemal nieograniczoną władzę'

## Zadanie 5. (5+1p) 
W zadaniu tym zajmiemy się kolokacjami słów, o których mamy informacje gramatyczną. Napisz program, który dla danego słowa znajduje k najbardziej z nim „spokrewnionych” słów. Rozważ następujące metody wyznaczania kolokacji:

a) PPMI (Positive Pointwise Mutual Information)

b) jakiś inny, dowolnie wybrany, z używanych na kolokacje wzorów (więcej na wykładzie),

c) kolokacje gramatyczno-słowowe (tzn. żeby dwa słowa były uznane za kolokacje, warunek kolokacyjności (dowolnie wybrany) powinny spełniać zarówno tagi słów, jak i same słowa.

d) jakaś dowolna inna metoda, lub Twoja modyfikacja powyższych (za to dodatkowy punkt).

Możesz się ograniczyć do słów, które są stosunkowo częste (więcej niż n wystąpień w korpusie)
i występują co najmniej raz w jakimś trigramie (lub k razy w jakimś bigramie). Wybierz niewielki
zbiór słów (powiedzmy koło 10). Przygotuj raport, w którym dla każdego z tych słów jest 10
najbardziej spokrewnionych słow

In [262]:
def PPMI(w1,w2):
    if w1 in BIGRAMS_DICT:
        Pw1 = sum(BIGRAMS_DICT[w1].values())
        if w2 in BIGRAMS_DICT[w1]:
            Pw1w2 = BIGRAMS_DICT[w1][w2]
        else:
            Pw1w2 = 1
    else:
        return 0
    if w2 in BIGRAMS_DICT:
        Pw2 = sum(BIGRAMS_DICT[w2].values())
    else:
        return 0
    N = 383345990
    if Pw1w2 == 0:
        return 0
    return max(0,math.log((N*Pw1w2)/(Pw1*Pw2)))  

In [263]:
def PPMI_alpha(w1,w2):
    if w1 in BIGRAMS_DICT:
        Pw1 = sum(BIGRAMS_DICT[w1].values())**0.75
        if w2 in BIGRAMS_DICT[w1]:
            Pw1w2 = (BIGRAMS_DICT[w1][w2])**0.75
        else:
            Pw1w2 = 1
    else:
        return 0
    if w2 in BIGRAMS_DICT:
        Pw2 = sum(BIGRAMS_DICT[w2].values())**0.75
    else:
        return 0
    N = 26544910
    if Pw1w2 == 0:
        return 0
    return max(0,math.log((N*Pw1w2)/(Pw1*Pw2)))  

In [264]:
def PSM(w1,w2):
    if w1 in BIGRAMS_DICT:
        Pw1 = sum(BIGRAMS_DICT[w1].values())
        if w2 in BIGRAMS_DICT[w1]:
            f = BIGRAMS_DICT[w1][w2]
        else:
            f = 1
    else:
        Pw1 = 1
    if w2 in BIGRAMS_DICT:
        Pw2 = sum(BIGRAMS_DICT[w2].values())
    else:
        Pw2 = 1
    
    N = 383345990
    f = max(1,f)
    f_dash = (Pw1*Pw2)/N
    return f*(math.log(f)-math.log(f_dash)-1)

In [265]:
BIGRAMS_TAGS_DICT = {}
with open('poleval_2grams.txt','r',encoding='UTF-8') as f:
    for line in f:
        n,s1,s2 = line.lower()[:-1].split(' ')
        t1,t2 = get_tag(s1),get_tag(s2)
        n = int(n)
        if n >= 10:
            if t1 not in BIGRAMS_TAGS_DICT:
                BIGRAMS_TAGS_DICT[t1] = {t2:n}
            elif t2 not in BIGRAMS_TAGS_DICT[t1]:
                BIGRAMS_TAGS_DICT[t1][t2] = n
            else:
                BIGRAMS_TAGS_DICT[t1][t2] += n

KeyboardInterrupt: 

In [304]:
def tags_PPMI(w1,w2):
    t1,t2=get_tag(w1),get_tag(w2)
    w_ppmi = PPMI(w1,w2)
    
    t_ppmi = -1
    if t1 in BIGRAMS_TAGS_DICT:
        Pt1 = sum(BIGRAMS_TAGS_DICT[t1].values())
        if t2 in BIGRAMS_TAGS_DICT[t1]:
            Pt1t2 = BIGRAMS_TAGS_DICT[t1][t2]
        else:
            Pt1t2 = 1
    else:
        t_ppmi = 0
    if t2 in BIGRAMS_TAGS_DICT:
        Pt2 = sum(BIGRAMS_TAGS_DICT[t2].values())
    else:
        t_ppmi = 0
    N = 383345990
    Pt1t2 = max(1,Pt1t2)
    if t_ppmi < 0:
        t_ppmi = max(0,math.log((N*Pt1t2)/(Pt1*Pt2)))  
    
    return t_ppmi * w_ppmi

In [315]:
TEST_WORDS = ['zaraz','jadę','bardzo','zielony','dobry','widze','promień','europejska','samolot']
methods = [('PPMI',PPMI),('PPMI, alpha=0.75',PPMI_alpha),('PSM',PSM),('PPMI with tags',tags_PPMI)]
for w in TEST_WORDS:
    print(f'Test word: {w}')
    for name,method in methods:
        print()
        print('Method:',name)
        scores = [(method(w,nw),nw) for nw in BIGRAMS_DICT[w].keys()]
        scores.sort(key=lambda e:e[0],reverse=True)
        for (score,nw) in scores[:10]:
            print(w,nw,score)
    print()
    print('-----------------------------------------------')
    print()

Test word: zaraz

Method: PPMI
zaraz skończę 7.570587474063663
zaraz wyjaśnię 7.545610190977466
zaraz udzielę 6.649804346735922
zaraz padnie 6.525250099209104
zaraz potem 6.295957889403607
zaraz przejdę 6.197812965359857
zaraz wrócę 6.11088021275798
zaraz kończę 6.079464016524601
zaraz idę 5.84772823655224
zaraz wracam 5.687771642151082

Method: PPMI, alpha=0.75
zaraz skończę 7.948952798448117
zaraz wyjaśnię 7.930219836133469
zaraz udzielę 7.258365452952312
zaraz padnie 7.164949767307197
zaraz potem 6.992980609953075
zaraz przejdę 6.919371916920262
zaraz wrócę 6.854172352468854
zaraz kończę 6.83061020529382
zaraz idę 6.6568083703145495
zaraz wracam 6.536840924513681

Method: PSM
zaraz po 26473.00154846656
zaraz potem 4353.277385089765
zaraz za 1276.2782921746407
zaraz powiem 933.3065014551216
zaraz obok 864.9702522985185
zaraz będę 283.6316421938258
zaraz przejdę 213.11033157975413
zaraz będzie 179.2027843513347
zaraz wyjaśnię 170.18586496541408
zaraz udzielę 158.19452170860586

Method

samolot szkolno-treningowy 11.917011543852835
samolot szturmowy 10.825319698029299
samolot an-2td 10.818399255184724
samolot an-2p 10.636077698390771
samolot wielozadaniowy 10.581161940794656
samolot patrolowy 10.033902296036551
samolot przewoził 9.985490132249621
samolot zwiadowczy 9.88409001780789
samolot bombowy 9.84041495430523
samolot odrzutowy 9.82852016665308

Method: PPMI, alpha=0.75
samolot szkolno-treningowy 11.208770850789996
samolot szturmowy 10.390001966422343
samolot an-2td 10.384811634288914
samolot an-2p 10.248070466693447
samolot wielozadaniowy 10.206883648496362
samolot patrolowy 9.796438914927784
samolot przewoził 9.760129792087586
samolot zwiadowczy 9.684079706256288
samolot bombowy 9.651323408629292
samolot odrzutowy 9.64240231789018

Method: PSM
samolot miał 1838.9089364269546
samolot został 1741.7203864829585
samolot myśliwski 1733.8187704402194
samolot wielozadaniowy 1188.0640806585375
samolot pasażerski 1166.3909002099451
samolot był 1112.04980670845
samolot tr

## Zadanie 6. (7p) 
W zadaniu tym będziemy tworzyć pierwszą wersję programu tworzącego poezję (przypominającą Pana Tadeusza, oznaczanego dalej PT)). Przypomnijmy najważniejsze fakty odnoszące się do tego utworu (i ogólnie Poezji (1)):

F1. Wiersz składa się z wersetów, z których każdy ma ustaloną liczbę sylab (w PT to 13)

F2. Ostatnie słowo w wersecie n rymuje się z ostatnim słowem w wersecie n + 1 (dla n parzystego,
numeracja od 0 (2)).

F3. Rym to zgodność ostatniej sylaby i części od samogłoski sylaby przedostatniej (zdrowie-dowie).
W zasadzie rymy to zjawisko fonetyczne, ale dla języka polskiego (w pierwszej wersji) można
sobie to trochę uprościć i powiedzieć, że dotyczą one liter.

F4. Akcenty słów i podziały słów muszą się jakoś sensownie układać. Nam wystarczy przyjąć,
że godzimy się jedynie na takie podziały wersu na słowa k-sylabowe (3), których użył Adam
Mickiewicz, na przykład:

Litwo, Ojczyzno moja, Ty jesteś jak zdrowie, ile cię trzeba cenić, ten tylko się dowie
ma schemat: [2,3,2,1,2,1,2] -- [2,1,2,2,1,2,1,2]

F5. Podział na sylaby nie jest trywialny (dlaczego?). Ale na nasze szczęście policzenie sylab jest
łatwe. Słowo ma tyle sylab, ile ma samogłosek (przy czym połączenia ie, iu, ię, itd traktujemy
jako jedną samogłoskę). Część rymowana wyrazu to część wyrazu od przedostatniej samogłoski
do końca.

Powinieneś stworzyć program, generujący dwuwersowe fragmenty wierszy w stylu PT, czyli powinieneś przypilnować:

a) żeby wersy były poprawne rytmicznie i się rymowały,

b) żeby dwuwers miał sens gramatyczny (czyli by tagi słów pasowały do jakiegoś zdania lub fragmentu zdania)

c) żeby były jakoś wykorzystane statystyki N-gra


---

(1) Bardzo proszę nie pokazywać tego żadnemu Poloniście ani Polonistce

(2) Jest to najprostszy schemat rymów aabb. Oczywiście są też inne, np. abab, ale na razie nimi się nie zajmujemy

(3) Korzystamy tu istotnie z tego, że z dobrym przybliżeniem, w języku polskim akcent wyrazowy jest funkcją liczby sylab


In [174]:
vowels = {'a', 'ą', 'e', 'ę', 'i', 'o', 'u', 'y', 'ó'}

def vowels_split(w):
    res = []
    i,j = -1,0
    while j < len(w):
        if w[j] in vowels:
            if i!= -1:   
                res.append(w[i:j])
            i = j
            if j+1<len(w) and w[j] == 'i' and w[j+1] in vowels:
                j+=1
        j+=1
        
    if i!= -1:
        res.append(w[i:])
        
    return res

In [192]:
regex = re.compile('[%s]' % re.escape(string.punctuation + '„”—»«'))

SCHEMES = defaultdict(int)
TAGS_SCHEMES = defaultdict(int)
skipped = 0

with open('pan-tadeusz.txt', 'r', encoding='utf-8') as f:
    for l in f:
        if len(l) < 20 or len(l) > 70:
            continue
        l = regex.sub('', l[:-1]).lower().replace('é', 'e')
        words = [w for w in l.split(' ') if len(w) > 0]
        volwes_counts = tuple(map(lambda w: len(vowels_split(w)), words))
        
        if sum(volwes_counts) != 13:
            skipped += 1
            continue
            
        SCHEMES[volwes_counts] += 1
        
        tags_scheme = tuple(map(get_tag,words))
        TAGS_SCHEMES[tags_scheme] += 1
        
        for i in range(1,len(words)):
            w1, w2 = words[i-1], words[i]
            t1, t2 = get_tag(w1), get_tag(w2)
            add_bigram(w1, w2)
            add_bigram_tag(t1,t2)

In [186]:
def scheme_pbb(scheme):
    return SCHEMES[scheme]/10000

In [202]:
VOWELS_TO_WORD = {}
for w in BIGRAMS_DICT.keys():
    vowels_count = len(vowels_split(w))
    if vowels_count not in VOWELS_TO_WORD:
        VOWELS_TO_WORD[vowels_count] = [w]
    else:
        VOWELS_TO_WORD[vowels_count].append(w)

In [228]:
REVERSE_BIGRAMS = {}
for k in BIGRAMS_DICT.keys():
    for v,n in BIGRAMS_DICT[k].items():
        if v not in REVERSE_BIGRAMS:
            REVERSE_BIGRAMS[v] = {k:n}
        elif k not in REVERSE_BIGRAMS[v]:
            REVERSE_BIGRAMS[v][k] = n
        else:
            REVERSE_BIGRAMS[v][k] += n

In [229]:
REVERSE_BIGRAMS_TAGS = {}
for k in BIGRAMS_TAGS.keys():
    for v,n in BIGRAMS_TAGS[k].items():
        if v not in REVERSE_BIGRAMS:
            REVERSE_BIGRAMS_TAGS[v] = {k:n}
        elif k not in REVERSE_BIGRAMS[v]:
            REVERSE_BIGRAMS_TAGS[v][k] = n
        else:
            REVERSE_BIGRAMS_TAGS[v][k] += n

In [233]:
def is_rhymes(w1, w2):
    return ''.join(vowels_split(w1)[-2:]) == ''.join(vowels_split(w2)[-2:])

In [260]:
def gen_verses():
    schemas, s_weights = list(SCHEMES.keys()), list(SCHEMES.values())
    v1_scheme = random.choices(schemas, weights=s_weights, k=1)[0]
    v2_scheme = random.choices(schemas, weights=s_weights, k=1)[0][::-1]
    
    total_scheme = v1_scheme + v2_scheme
    s1_end_index = len(v1_scheme)-1

    K = 1000
    i = 1
    res = []
    current_sentences = [[w] for w in VOWELS_TO_WORD[v1_scheme[0]]]
    next_sentences = []
    while i < len(v1_scheme):

#         print(i, len(current_sentences))
        if len(current_sentences) > K:
            random.shuffle(current_sentences)
            current_sentences = current_sentences[:K]

        searched = {}

        for sent in current_sentences:

            if sent[-1] in searched:
                for w in searched[sent[-1]]:
                    new_sent = sent[:]
                    new_sent.append(w)
                    next_sentences.append(new_sent)
                continue

            if sent[-1] not in BIGRAMS_DICT:
                continue
            D = BIGRAMS_DICT[sent[-1]]

            t1 = get_tag(sent[-1])
            if t1 not in BIGRAMS_TAGS:
                continue
            D_tags = BIGRAMS_TAGS[t1]

            sw = []
            for pnw in VOWELS_TO_WORD[v1_scheme[i]]:
                if pnw in D and get_tag(pnw) in D_tags:
                    new_sent = sent[:]
                    new_sent.append(pnw)
                    sw.append(pnw)
                    next_sentences.append(new_sent)

            if len(sw)>0:
                searched[sent[-1]] = sw

        if len(next_sentences) == 0:
            choosen_sent = random.choice(current_sentences)
            res += choosen_sent
            current_sentences = [[w] for w in VOWELS_TO_WORD[v1_scheme[i]]]
        else:
            current_sentences = next_sentences
            next_sentences = []
        i += 1
        
#     print('Szukam rymu...')
    next_sentences = []
    next_rhymes = list(map(lambda w: ''.join(vowels_split(w)[-2:]), VOWELS_TO_WORD[v2_scheme[0]]))
    for sent in current_sentences:
        sent_rhyme = ''.join(vowels_split(sent[-1])[-2:])
        if len(next_sentences) > K:
            break
        for i, w in enumerate(VOWELS_TO_WORD[v2_scheme[0]]):
            if len(next_sentences) > K:
                break
            if sent_rhyme == next_rhymes[i]:
                new_sent = sent[:]
                new_sent.append(w)
                next_sentences.append(new_sent)
                
    i = 1
    current_sentences = next_sentences
    next_sentences = []
    while i < len(v2_scheme):
        
#         print(i+len(v1_scheme), len(current_sentences))
        if len(current_sentences) > K:
            random.shuffle(current_sentences)
            current_sentences = current_sentences[:K]

        searched = {}

        for sent in current_sentences:

            if sent[-1] in searched:
                for w in searched[sent[-1]]:
                    new_sent = sent[:]
                    new_sent.append(w)
                    next_sentences.append(new_sent)
                continue

            if sent[-1] not in REVERSE_BIGRAMS:
                continue
            D = REVERSE_BIGRAMS[sent[-1]]

            t1 = get_tag(sent[-1])
            if t1 not in REVERSE_BIGRAMS_TAGS:
                continue
            D_tags = REVERSE_BIGRAMS_TAGS[t1]

            sw = []
            for pnw in VOWELS_TO_WORD[v2_scheme[i]]:
                if pnw in D and get_tag(pnw) in D_tags:
                    new_sent = sent[:]
                    new_sent.append(pnw)
                    sw.append(pnw)
                    next_sentences.append(new_sent)

            if len(sw)>0:
                searched[sent[-1]] = sw

        if len(next_sentences) == 0:
            choosen_sent = random.choice(current_sentences)
            res += choosen_sent
            current_sentences = [[w] for w in VOWELS_TO_WORD[v2_scheme[i]]]
        else:
            current_sentences = next_sentences
            next_sentences = []
        i += 1
    
    choosen_sent = random.choice(current_sentences)
    res += choosen_sent
    
    print(' '.join(res[:s1_end_index+1]))
    print(' '.join(res[s1_end_index+1:][::-1]))

In [261]:
gen_verses()

skąd ono innych śląskich oraz wszystkim może
zmieszał pa pa pa czołgi alf mrozu szczęść boże


## Zadanie 7. (5p) 
Zmodyfikuj algorytm z poprzedniego zadania w ten sposób, by starał się on maksymalizować wzajemną „kolokacyjność” słów z dwuwersu (tak, by jak najwięcej słów było ze sobą powiązanych, na przykład przez wysokie PPMI). Akceptowalna jest dowolna procedura (local search, jakieś błądzenie losowe, metody ewolucyjne, ...), która daje wartość liczby par słów będących kolokacjami istotnie większą niż losowanie z poprzedniego zadania.

In [284]:
def gen_verses_2():
    schemas, s_weights = list(SCHEMES.keys()), list(SCHEMES.values())
    v1_scheme = random.choices(schemas, weights=s_weights, k=1)[0]
    v2_scheme = random.choices(schemas, weights=s_weights, k=1)[0][::-1]
    
    total_scheme = v1_scheme + v2_scheme
    s1_end_index = len(v1_scheme)-1

    K = 1000
    i = 1
    res = []
    current_sentences = [[0,w] for w in VOWELS_TO_WORD[v1_scheme[0]]]
    next_sentences = []
    while i < len(v1_scheme):

        #print(i, len(current_sentences))
        current_sentences.sort(key=lambda e: e[0], reverse=True)
        #print(current_sentences[0][0], current_sentences[1][0], np.mean(np.array([a[0] for a in current_sentences])))
        if len(current_sentences) > K:
            current_sentences = current_sentences[:K]

        searched = {}

        for sent in current_sentences:

            if sent[-1] in searched:
                for w in searched[sent[-1]]:
                    new_sent = sent[:]
                    new_sent.append(w)
                    new_sent[0] += PPMI(sent[-1], w)
                    next_sentences.append(new_sent)
                continue

            if sent[-1] not in BIGRAMS_DICT:
                continue
            D = BIGRAMS_DICT[sent[-1]]

            t1 = get_tag(sent[-1])
            if t1 not in BIGRAMS_TAGS:
                continue
            D_tags = BIGRAMS_TAGS[t1]

            sw = []
            for pnw in VOWELS_TO_WORD[v1_scheme[i]]:
                if pnw in D and get_tag(pnw) in D_tags:
                    new_sent = sent[:]
                    new_sent.append(pnw)
                    new_sent[0] += PPMI(sent[-1], pnw)
                    sw.append(pnw)
                    next_sentences.append(new_sent)

            if len(sw)>0:
                searched[sent[-1]] = sw

        if len(next_sentences) == 0:
            choosen_sent = current_sentences[0][1:]
            res += choosen_sent
            current_sentences = [[PPMI(res[-1], w), w] for w in VOWELS_TO_WORD[v1_scheme[i]]]
        else:
            current_sentences = next_sentences
            next_sentences = []
        i += 1
        
    # print('Szukam rymu...')
    next_sentences = []
    next_rhymes = list(map(lambda w: ''.join(vowels_split(w)[-2:]), VOWELS_TO_WORD[v2_scheme[0]]))
    current_sentences.sort(key=lambda e: e[0], reverse=True)
    for sent in current_sentences:
        sent_rhyme = ''.join(vowels_split(sent[-1])[-2:])
        if len(next_sentences) > K:
            break
        for i, w in enumerate(VOWELS_TO_WORD[v2_scheme[0]]):
            if len(next_sentences) > K:
                break
            if sent_rhyme == next_rhymes[i]:
                new_sent = sent[:]
                new_sent.append(w)
                next_sentences.append(new_sent)
                
    i = 1
    current_sentences = next_sentences
    next_sentences = []
    while i < len(v2_scheme):
#         print(i+len(v1_scheme), len(current_sentences))
        current_sentences.sort(key=lambda e: e[0], reverse=True)
        # print(current_sentences[0][0], current_sentences[1][0], np.mean(np.array([a[0] for a in current_sentences])))
        if len(current_sentences) > K:
            current_sentences = current_sentences[:K]

        searched = {}

        for sent in current_sentences:

            if sent[-1] in searched:
                for w in searched[sent[-1]]:
                    new_sent = sent[:]
                    new_sent.append(w)
                    new_sent[0] += PPMI(w, sent[-1])
                    next_sentences.append(new_sent)
                continue

            if sent[-1] not in REVERSE_BIGRAMS:
                continue
            D = REVERSE_BIGRAMS[sent[-1]]

            t1 = get_tag(sent[-1])
            if t1 not in REVERSE_BIGRAMS_TAGS:
                continue
            D_tags = REVERSE_BIGRAMS_TAGS[t1]

            sw = []
            for pnw in VOWELS_TO_WORD[v2_scheme[i]]:
                if pnw in D and get_tag(pnw) in D_tags:
                    new_sent = sent[:]
                    new_sent.append(pnw)
                    new_sent[0] += PPMI(pnw, sent[-1])
                    sw.append(pnw)
                    next_sentences.append(new_sent)

            if len(sw)>0:
                searched[sent[-1]] = sw

        if len(next_sentences) == 0:
            choosen_sent = current_sentences[0][1:]
            res += choosen_sent
            current_sentences = [[PPMI(w, res[-1]), w] for w in VOWELS_TO_WORD[v2_scheme[i]]]
        else:
            current_sentences = next_sentences
            next_sentences = []
        i += 1
    
    current_sentences.sort(key=lambda e: e[0], reverse=True)
    choosen_sent = current_sentences[0][1:]
    res += choosen_sent
    
    print(' '.join(res[:s1_end_index+1]))
    print(' '.join(res[s1_end_index+1:][::-1]))

In [285]:
gen_verses_2()

ojczysty rzekł kichnąwszy skołuba dobrodziej
litwo kroćset kroci i 4244 litwo zacz kołodziej


In [286]:
ZDANIA = [['nasz', 'przedmiot', 'to', 'przetwarzanie', 'języka', 'naturalnego'],
 ['jedną', 'grupę', 'prowadzi', 'jakub', 'michaliszyn'],
 ['wykładowcą', 'jest', 'paweł', 'rychlikowski'],
 ['pan', 'paweł', 'prowadzi', 'również', 'dwie', 'pozostałe', 'grupy'],
 ['na', 'stronie', 'wykładu', 'będą', 'pojawiać', 'się', 'listy', 'zadań'],
 ['będą', 'to', 'listy', 'ćwiczeniowe', 'i', 'pracowniowe'],
 ['będą', 'to', 'listy', 'pracowniowe', 'i', 'ćwiczeniowe'],
 ['mogą', 'się', 'pojawiać', 'również', 'dodatkowe', 'aktywności', ''],
 ['te', 'aktywności', 'będą', 'premiowane', ''],
 ['na', 'przykłąd', 'quizy', 'na', 'platformie', 'zdalnej'],
 ['rozważamy', 'też', 'miniprojekty', 'na', 'temat', 'wykładu'],
 ['zapowiadamy', 'je', 'z', 'co', 'najmniej', 'tygodniowym', 'wyprzedzeniem'],
 ['zadania', 'są', 'przeznaczone', 'na', 'konkretne', 'zajęcia'],
 ['ćwiczenia', 'zaczynają', 'się', 'w', 'sali', 'seminaryjnej'],
 ['jest', 'tak', 'tylko', 'w', 'trybie', 'stacjonarnym', ''],
 ['pozostałe', 'zajęcia', 'domyślnie', 'zaczynają', 'się', 'w', 'pracowni'],
 ['pracownia', 'polega', 'na', 'indywidualnych', 'konsutlacjach'],
 ['rozmawiamy', 'tam', 'na', 'temat', 'rozwiązań', ''],
 ['podczas',
  'ćwiczeń',
  'studenci',
  'przedstawiają',
  'pisemne',
  'rozwiązania',
  'zadań'],
 ['tryb', 'zajęć', 'zależy', 'od', 'sytuacji', 'epidemiologicznej'],
 ['zależy', 'też', 'od', 'wymogów', 'formalnych', ''],
 ['zależy', 'również', 'od', 'decyzji', 'władz'],
 ['uwzględnia', 'się', 'możliwości', 'studentów', 'i', 'prowadzących'],
 ['może', 'się', 'on', 'zmieniać', 'w', 'trakcie', 'semestru'],
 ['przed',
  'ćwiczeniami',
  'studenci',
  'dostarczają',
  'prowadzącemu',
  'deklaracje'],
 ['zawiera', 'ona', 'numery', 'zadań'],
 ['studenci', 'pokazują', 'rozwiązania', 'przy', 'tablicy'],
 ['jest', 'źle', ',', 'jeżeli', 'student', 'nie', 'umie'],
 ['dodatkowo', 'coś', 'mu', 'się', 'odejmuje'],
 ['zadania', 'można', 'oddawać', 'podczas', 'zajęć', 'na', 'pracowni'],
 ['zadania', 'należy', 'rozwiązywać', 'samodzielnie'],
 ['prowadzący', 'ocenia', 'poprawność', 'rozwiązania'],
 ['czasem', 'mogą', 'pojawić', 'się', 'informacje', 'o', 'regułach'],
 ['na', 'każdej', 'liście', 'podany', 'jest', 'termin'],
 ['za', 'poprawne', 'rozwiązanie', 'dostajemy', 'dużo', 'punktów'],
 ['spóźnione', 'zadania', 'nie', 'będą', 'sprawdzane'],
 ['pierwsza', 'lista', 'ma', 'dłuższy', 'termin'],
 ['zasada', 'musi', 'być', 'zmodyfikowana', 'pod', 'koniec', 'semestru'],
 ['w', 'przypadku', 'choroby', 'studenta', 'obowiązują', 'inne', 'zasady'],
 ['w', 'przypadku', 'kwarantanny', 'studenta', 'obowiązują', 'inne', 'zasady'],
 ['w', 'przypadku', 'izolacji', 'studenta', 'obowiązują', 'inne', 'zasady'],
 ['możliwe',
  'jest',
  'oddawanie',
  'zaległych',
  'zadań',
  'bez',
  'straty',
  'punktowej'],
 ['ocenę', 'wystawia', 'się', 'zgodnie', 'z', 'tabelką'],
 ['50', 'punktów', 'daje', 'ocenę', 'dostateczną'],
 ['120', 'punktów', 'daje', 'ocenę', 'bardzo', 'dobrą'],
 ['powyższa', 'tabelka', 'zostanie', 'przeskalowana'],
 ['może', 'też', 'zostać', 'przeskalowana', 'z', 'innych', 'powodów'],
 ['w', 'pierwszym', 'tygodniu', 'zajęć', 'coś', 'podamy'],
 ['dodatkowe', 'zasady', 'umożliwiające', 'zwolnienie', 'z', 'egzaminu'],
 ['dotyczą', 'osób', 'z', 'dużą', 'liczbą', 'punktów'],
 ['można', 'rozwiązać', 'też', 'zadanie', 'specjalne'],
 ['forma', 'egzaminu', 'zależy', 'od', 'wielu', 'czynników'],
 ['należy', 'się', 'spodziewać', 'różnych', 'wariantów'],
 ['nie', 'wykluczamy', 'indywidualnego', 'egzaminu', 'ustnego'],
 ['jaś', 'i', 'małgosia', 'też', 'są', 'tutaj'],
 ['puchatek', 'z', 'prosiaczkiem', 'nie', 'przybędą'],
 ['raz', 'kotek', 'był', 'chory', 'i', 'odpoczywał'],
 ['raz', 'kotek', 'był', 'chory', 'i', 'leżał', 'w', 'łóżeczku'],
 ['definicja', 'relacji', 'równoważności', 'nie', 'jest', 'trudna'],
 ['hipoteza', 'continuum', 'jest', 'niezależna', 'od', 'aksjomatów'],
 ['teoria', 'mnogości', 'daje', 'wiele', 'radości'],
 ['programowanie', 'nie', 'jest', 'szczególnie', 'trudne'],
 ['rachunek', 'prawdopodobieństwa', 'przydaje', 'się', 'w', 'życiu'],
 ['rycerze', 'osaczyli', 'smoka', 'zanim', 'strawił', 'księżniczkę'],
 ['chyba', 'należy', 'już', 'przestać', 'wymyślać', 'zdania']]